# docker-compose 的安裝

In [ ]:
sudo curl -L "https://github.com/docker/compose/releases/download/1.29.2/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose

In [ ]:
sudo chmod +x /usr/local/bin/docker-compose

In [ ]:
docker-compose --version

# 什麼是 Docker Compose

# docker-compose 的文件結構和版本

docker-compose 參考網站: https://docs.docker.com/compose/compose-file/ <br>
docker-compose 語法版本: https://docs.docker.com/compose/compose-file/compose-versioning/

In [ ]:
version: "3.8"
    
services: # 容器
  servicename: # 服務的名字，對應 docker container run 的長選項 --name，這個名字也是內部 bridge 網路可使用的 DNS 名字。
    image: # 鏡像的名字。
    command: # 可選，如果設置會覆蓋默認鏡像裡面的 CMD 命令。
    environment: # 可選，相當於 docker container run 裡面的長選項 --env。
    volumes: # 可選，相當於 docker container run 裡面的短選項 -v。
    networks: # 可選，相當於 docker container run 裡面的長選項 --network。
    ports: # 可選，相當於 docker container run 裡面的短選項 -p。
  servicename:
    ...
    
volumes: # 可選，相當於 docker volume create。
networks: # 可選，相當於 docker network create。

### 範例01-Python Flask + Redis

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket
import redis

app = Flask(__name__)
redis = Redis(host=os.environ.get("REDIS_HOST", "127.0.0.1"), port=6379)


@app.route("/")
def index():
    redis.incr("hits")
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask redis && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src &&\
    chown -R flask:flask /src

USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK_APP=app.py REDIS_HOST=redis FLASK_RUN_HOST=0.0.0.0
EXPOSE 5000
CMD ["flask", "run"]

In [ ]:
# prepare image
docker image pull redis
docker image build -t flask-app .

# create network
docker network create -d bridge demo-network

# create container
docker container run -d --name redis-server --network demo-network redis
docker container run -d --network demo-network --name flask-demo --env REDIS_HOST=redis-server -p 5050:5000 flask-app

# docker-compose 命令的基本操作

In [ ]:
docker image build -t flask-app .
docker image pull redis:latest

In [ ]:
docker-compose up 

In [ ]:
docker-compose ps
docker container ls -a
docker network ls

In [ ]:
docker-compose rm

In [ ]:
docker-compose up -d 

In [ ]:
docker-compose logs -f

# docker-compose 鏡像的建構與拉取

In [ ]:
version: "3.8"

services:
  flask-demo:
    build: 
      context: ./flask
      dockerfile: Dockerfile 
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
    networks:
      - demo-network
    ports:
      - 5050:5000

  redis-server:
    image: redis:latest
    networks:
      - demo-network

networks:
  demo-network:

In [ ]:
docker-compose build
docker image ls

In [ ]:
docker-compose pull

# docker-compose 服務的更新

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket
import redis

app = Flask(__name__)
redis = Redis(host=os.environ.get("REDIS_HOST", "127.0.0.1"), port=6379)


@app.route("/")
def index():
    redis.incr("hits")
    return f"Hello Docker World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

@app.route("/add")
def add():
    return "Hello, Matt!"

In [ ]:
docker-compose up -d --build

In [ ]:
version: "3.8"

services:
  flask-demo:
    build: 
      context: ./flask
      dockerfile: Dockerfile 
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
    networks:
      - demo-network
    ports:
      - 8080:5000

  redis-server:
    image: redis:latest
    networks:
      - demo-network

  busybox:
    image: busybox:latest
    command: sh -c "while true; do sleep 3600; done"
    networks:
      - demo-network

networks:
  demo-network:

In [ ]:
docker-compose up -d 

In [ ]:
docker-compose up -d 

In [ ]:
docker-compose up -d --remove-orphans

# docker-compose up/down 與 restart 的差異

# docker-compose 網路: 使用預設網路

In [ ]:
version: '3.8'

services:
  box1:
    image: xiaopeng163/net-box:latest
    command: /bin/sh -c "while true; do sleep 3600; done"

  box2:
    image: xiaopeng163/net-box:latest
    command: /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker-compose up -d 

In [ ]:
docker network ls
docker network inspect demo22_default

In [ ]:
docker container exec -it demo22_box1_1 sh
$ ping demo22_box2_1
$ ping box2

In [ ]:
dig
more /etc/resolv.conf

In [ ]:
ping box1
more /etc/resolv.conf

In [ ]:
docker container exec -it demo22_box1_1 sh
$ ping www.yahoo.com.tw

# docker-compose 網路: 使用自定義網路

In [ ]:
version: '3.8'

services:
  box1:
    image: xiaopeng163/net-box:latest
    command: /bin/sh -c "while true; do sleep 3600; done"
    networks:
      - mynetwork1

  box2:
    image: xiaopeng163/net-box:latest
    command: /bin/sh -c "while true; do sleep 3600; done"
    networks:
      - mynetwork1
      - mynetwork2

networks:
  mynetwork1:
    ipam:
     driver: default
     config:
      - subnet: "172.16.238.0/24"
  mynetwork2:
    ipam:
     driver: default

In [ ]:
docker-compose up -d
docker netowrk ls
docker network inspect demo23_mynetwork1
docker network inspect demo23_mynetwork2
docker container inspect demo23_box1_1
docker container inspect demo23_box2_1

# Web Server 簡單的介紹

### 什麼是 web server

<img src="img/16.png">

### 程式語言不是也可以起一個 web server 嗎

### 那 Application Server 跟 Web Server 的差異又為何呢

### 正向代理與反向代理

<img src="img/17.png">

# nginx 簡單的介紹

<img src="img/18.png">

### 面向效能

### 設置簡易

### 記憶體消耗低

### 反向代理＆負載平衡

# 水平擴展與負載均衡

### 範例01

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket


app = Flask(__name__)
redis = Redis(host=os.environ.get('REDIS_HOST', '127.0.0.1'), port=6379)


@app.route('/')

def hello():
    redis.incr('hits')
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask redis && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src && \
    chown -R flask:flask /src
USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK=app.py REDIS_HOST=redis
EXPOSE 5000
CMD ["flask", "run", "-h", "0.0.0.0"]

In [ ]:
version: "3.8"

services:
  flask:
    build:
      context: ./flask
      dockerfile: Dockerfile
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
  redis-server:
    image: redis:latest
  client:
    image: xiaopeng163/net-box:latest
    command: sh -c "while true; do sleep 3600; done;"

In [ ]:
docker-compose up -d

In [ ]:
docker-compose up -d --scale flask=3
docker-compose ps

In [ ]:
docker-compose up -d --scale flask=3
docker container exec -it clientId sh

In [ ]:
ping flask 
ping flask 
ping flask 
ping flask 
ping flask 
curl flask:5000
curl flask:5000
curl flask:5000
curl flask:5000
curl flask:5000
docker-compose down

### 範例02

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket


app = Flask(__name__)
redis = Redis(host=os.environ.get('REDIS_HOST', '127.0.0.1'), port=6379)


@app.route('/')
def hello():
    redis.incr('hits')
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask redis && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src && \
    chown -R flask:flask /src
USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK=app.py REDIS_HOST=redis
EXPOSE 5000
CMD ["flask", "run", "-h", "0.0.0.0"]

In [ ]:
server {
  listen  80 default_server;
  location / {
    proxy_pass http://flask:5000;
  }
}

In [ ]:
version: "3.8"

services:
  flask:
    build:
      context: ./flask
      dockerfile: Dockerfile
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
    networks:
      - backend
      - frontend
  redis-server:
    image: redis:latest
    networks:
      - backend
  nginx:
    image: nginx:stable-alpine
    ports:
      - 8000:80
    depends_on:
      - flask
    volumes:
      - ./nginx/nginx.conf:/etc/nginx/conf.d/default.conf:ro
      - ./var/log/nginx:/var/log/nginx
    networks:
      - frontend
networks:
  backend:
  frontend:

In [ ]:
docker-compose up -d 
docker-compose ps
docker-compose up -d --scale flask=3
docker-compose ps

# 環境變量的設定

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket



app = Flask(__name__)
redis = Redis(host=os.environ.get('REDIS_HOST', '127.0.0.1'), port=6379,
              password=os.environ.get("REDIS_PASS"))

@app.route('/')
def hello():
    redis.incr('hits')
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask redis && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src && \
    chown -R flask:flask /src
USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK=app.py REDIS_HOST=redis
EXPOSE 5000
CMD ["flask", "run", "-h", "0.0.0.0"]

In [ ]:
server {
  listen  80 default_server;
  location / {
    proxy_pass http://flask:5000;
  }
}

In [ ]:
REDIS_PASSWORD=abc123

In [ ]:
version: "3.8"

services:
  flask:
    build:
      context: ./flask
      dockerfile: Dockerfile
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
      - REDIS_PASS=${REDIS_PASSWORD}
    networks:
      - backend
      - frontend
  redis-server:
    image: redis:latest
    command: redis-server --requirepass ${REDIS_PASSWORD}
    networks:
      - backend
  nginx:
    image: nginx:stable-alpine
    ports:
      - 8000:80
    depends_on:
      - flask
    volumes:
      - ./nginx/nginx.conf:/etc/nginx/conf.d/default.conf:ro
      - ./var/log/nginx:/var/log/nginx
    networks:
      - frontend
networks:
  backend:
  frontend:

In [ ]:
docker-compose config
docker-compose up -d --scale flask=3

# 服務依賴與健康檢查

https://gist.github.com/phuysmans/4f67a7fa1b0c6809a86f014694ac6c3a

### 範例01

In [ ]:
from flask import Flask
from redis import StrictRedis
import os
import socket



app = Flask(__name__)
redis = StrictRedis(host=os.environ.get('REDIS_HOST', '127.0.0.1'),
                    port=6379, password=os.environ.get('REDIS_PASS'))


@app.route('/')
def hello():
    redis.incr('hits')
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim

RUN pip install flask redis && \
    apt-get update && \
    apt-get install -y curl && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src && \
    chown -R flask:flask /src
USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK_APP=app.py REDIS_HOST=redis
EXPOSE 5000
CMD ["flask", "run", "-h", "0.0.0.0"]

In [ ]:
server {
  listen  80 default_server;
  location / {
    proxy_pass http://flask:5000;
  }
}

In [ ]:
version: "3.8"

services:
  flask:
    build:
      context: ./flask
      dockerfile: Dockerfile
    image: flask-demo:latest
    environment:
      - REDIS_HOST=redis-server
      - REDIS_PASS=${REDIS_PASSWORD}
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:5000"]
      interval: 30s
      timeout: 3s
      retries: 3
      start_period: 40s
    depends_on:
      redis-server:
        condition: service_healthy
    networks:
      - backend
      - frontend
  redis-server:
    image: redis:latest
    command: redis-server --requirepass ${REDIS_PASSWORD}
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]
      interval: 1s
      timeout: 3s
      retries: 30
    networks:
      - backend
  nginx:
    image: nginx:stable-alpine
    ports:
      - 8000:80
    depends_on:
      flask:
        condition: service_healthy
    volumes:
      - ./nginx/nginx.conf:/etc/nginx/conf.d/default.conf:ro
      - ./var/log/nginx:/var/log/nginx
    networks:
      - frontend
networks:
  backend:
  frontend:

# 綜合演練: docker-compose 投票 app 練習

 https://github.com/dockersamples/example-voting-app